# Build Convolutional Neural Network

<font color='steelblue'>

<span style="font-family:verdana; font-size:1.6em;">
    <strong>MNIST Digit Recognition</strong><br><br>
    From the Keras datasets, import the MNIST Digits data.<br>
    There are images of digits 0 to 9 and have labels associated
    with each image.<br>
</span>
<span style="font-family:verdana; font-size:1.4em;"><br>
    <b>Following examples are included in the processing:</b>
    <ol>
        <li>Check the version of Tensorflow and Keras </li>
        <li>Load training and test data including labels</li>
        <li>Normalize the images</li>
        <li>Plot few images after being normalized</li>
        <li>Create a Neural Network and build a model</li>
        <li>Train the model on the training dataset</li>
        <li>Evaluate the accuracy of the model using test dataset</li>
        <li>Plot the accuracy and loss for the model</li>
    </ol>    
</span>

</font>

In [ ]:
%config IPCompleter.greedy = True

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

In [ ]:
# make sure tensorflow is properly installed
tf.__version__, tf.keras.__version__

## Locate the dataset

In [ ]:
digits_mnist = keras.datasets.mnist

In [ ]:
(train_images, train_labels), (test_images, test_labels) = \
                                digits_mnist.load_data()

## Explore Data

In [ ]:
print("Size: train images {}, train labels {}".format(train_images.shape, 
                                                      train_labels.shape))

# save the number of items in training dataset
train_rows = train_images.shape[0]

In [ ]:
print("Size: test images {}, test labels {}".format(test_images.shape, 
                                                      test_labels.shape))

# save the number of items in test dataset
test_rows = test_images.shape[0]

In [ ]:
# look at first 10 labels in training set
train_labels[:10]

In [ ]:
plt.figure
plt.imshow(train_images[5], cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
train_images[0]

In [ ]:
# view 25 of the grayscale images

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)    # print 5 images per row
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()

## Normalize the images

In [ ]:
# Normalize the values to be between 0 and 1; min-max normalization

train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
train_images[0]

## Plot few normalized images

In [ ]:
# view 25 of the grayscale images

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)    # print 5 images per row
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()

In [ ]:
# define the image shape (horizontal pixels x vertical pixels x grey scale)
iShape = (28,28, 1)

In [ ]:
# The data needs to be reshaped.
# The first convolution expects a single tensor containing everything, 
# so instead of 60,000 28x28x1 items in an array (60000, 28, 28), 
# it wants a single 4D array/tensor that is 60000x28x28x1, otherwise you will get an error.

train_images = train_images.reshape(train_rows, 28, 28, 1)

# reshape test images into a single tensor
test_images = test_images.reshape(test_rows, 28, 28, 1)

# Create Convolution Neural Network

In [ ]:
# Create Convolution Neural Network - adding layers

model = keras.models.Sequential()
model.add(layers.Conv2D(32, activation = 'relu', input_shape = iShape, kernel_size=(3, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))   # (3,3) - filter size, 64 number of filters
model.add(layers.MaxPooling2D(pool_size=(2, 2)))          # (2,2) - pooling size

In [ ]:
# To avoid over fitting on training dataset, randomly drop neurons and their connections
# remove 25% of them
model.add(layers.Dropout(rate = 0.25))

In [ ]:
# Convert the previous layer into 1 dimensional array (flatten it)
model.add(layers.Flatten())

In [ ]:
# Once we’ve flattened the data into a 1D array, add a dense hidden layer, which is normal 
# to a traditional neural network. Next, add another dropout layer before 
# adding a final dense layer which classifies the data
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))   # helps classify into classes

In [ ]:
# Compile the model with chosen parameters
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

# Model Summary
<h3>
    <ul>
        <li>Shows all parameters and shape for each of the layers</li>
        <li>Trainable params: is total number of parameters in our model</li>
        <li>Non-trainable params: are parameters that cannot be used by model for training</li>
    </ul>
</h3>

# Train Convolutional Neural Network (CNN)
<h3><b>Used 10 epochs and got about 99.08% accuracy on training and the validation set</b></h3>

In [ ]:
# To measure time required to train
import timeit, time
start = timeit.default_timer()

In [ ]:
# Train the model and include a validation set (composed of 10% of the dataset)
# Capturing the returned history enables you to plot the change in 
# error/loss and accuracy over time
batches = 1000
epochs = 6
history = model.fit(train_images, train_labels, validation_split = 0.1, 
                    batch_size = batches, epochs = epochs)

In [ ]:
stop = timeit.default_timer()
execution_time = stop - start
exectime = time.strftime("%M:%S", time.gmtime(execution_time)) 
print("To train it took: {} mins".format(exectime))

In [ ]:
metrics_names = model.metrics_names

# Evaluate the accuracy of the model

In [ ]:
# Use the test images to evaluate the model on a set of unseen images

test_loss, test_acc = model.evaluate(test_images, test_labels)

print("Test accuracy: ", test_acc)

In [ ]:
# Function to plot the accuracy and loss
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.title('Training and validation')
    plt.xlabel('Epochs')
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [ ]:
for name in metrics_names:
    plot_graphs(history, name)

<span style="font-family:Comic sans MS; font-size:1.4em;">
<font color='tomato'>
    <h2>Practice</h2>
    <h3>Try out different parameters and see how model accuracy changes</h3>
    <ol>
        <li>Play with different epoch values (10, 20, ...)</li>
        <li>Add more Conv2D and Pooling layers</li>
        <li>Change number of neuron in each dense layer</li>
        <li>Change the batch size and see what happens</li>    
    </ol>
</font>
</span>